## Case 1: Underactuated Inverted Pendulum with Reaction Wheel


In [ ]:
import sympy as sp
from sympy.physics.mechanics import dynamicsymbols, init_vprinting
from IPython.display import display, Math
import matplotlib.pyplot as plt

init_vprinting()

We'll simplify the problem to one involving a reaction wheel at the top of an inverted pendulum and define our system as follows:

<img src="../assets/case-1.jpg" width=750/>


Next, we'll define the following variables to describe each joint and link in the system.


In [ ]:
# Constants
g, t = sp.symbols('g,t')

# Link 1
alpha_1 = 0
b_1 = 0
theta_1 = dynamicsymbols("theta_1")
theta_1d = sp.diff(theta_1, t)
theta_1dd = sp.diff(theta_1d, t)
d_1 = 0

l_1, l_c1 = sp.symbols("l_1 l_c1")

m_1 = sp.symbols("m_1")
I_c1 = sp.Rational(1, 12) * m_1 * l_1 ** 2
# I_c1 = 0

# Link 2
alpha_2 = 0
b_2 = l_1
theta_2 = dynamicsymbols("theta_2")
theta_2d = sp.diff(theta_2, t)
theta_2dd = sp.diff(theta_2d, t)
d_2 = 0

r_w, l_c2 = sp.symbols("r_w, l_c2")
m_2 = sp.symbols("m_2")
I_c2 = sp.Rational(1, 2) * m_2 * r_w ** 2


We can then calculate the transformation matrices for the different frames to the base frame.


In [ ]:
T_01 = sp.simplify(sp.Matrix([
    [sp.cos(theta_1), -sp.sin(theta_1), 0, b_1],  # type: ignore
    [sp.cos(alpha_1) * sp.sin(theta_1), sp.cos(alpha_1) * sp.cos(theta_1), -sp.sin(alpha_1), -d_1 * sp.sin(alpha_1)],  # type: ignore
    [sp.sin(alpha_1) * sp.sin(theta_1), sp.sin(alpha_1) * sp.cos(theta_1), sp.cos(alpha_1), d_1 * sp.cos(alpha_1)],  # type: ignore
    [0, 0, 0, 1]
]))

T_12 = sp.simplify(sp.Matrix([
    [sp.cos(theta_2), -sp.sin(theta_2), 0, b_2],  # type: ignore
    [sp.cos(alpha_2) * sp.sin(theta_2), sp.cos(alpha_2) * sp.cos(theta_2), -sp.sin(alpha_2), -d_2 * sp.sin(alpha_2)],  # type: ignore
    [sp.sin(alpha_2) * sp.sin(theta_2), sp.sin(alpha_2) * sp.cos(theta_2), sp.cos(alpha_2), d_2 * sp.cos(alpha_2)],  # type: ignore
    [0, 0, 0, 1]
]))

R_01 = T_01[:3, :3]
R_12 = T_12[:3, :3]
R_02 = R_01 @ R_12

D_01 = T_01[:3, 3]
D_12 = T_12[:3, 3]
D_2p = sp.Matrix([1, 0, 0]).reshape(3, 1)

D_1c1 = sp.Matrix([l_c1, 0, 0]).reshape(3, 1)

D_1c2 = D_12
D_2c2 = sp.Matrix([0, 0, 0]).reshape(3, 1)


In [ ]:
display(Math(f"T_0^1 = {sp.latex(T_01)}"))
display(Math(f"T_1^2 = {sp.latex(T_12)}"))

display(Math(f"R_0^1 = {sp.latex(R_01)}"))
display(Math(f"R_1^2 = {sp.latex(R_12)}"))
display(Math(f"R_0^2 = {sp.latex(R_02)}"))

display(Math(f"D_1^{{c1}} = {sp.latex(D_1c1)}"))
display(Math(f"D_1^{{c2}} = {sp.latex(D_1c2)}"))
display(Math(f"D_2^{{c2}} = {sp.latex(D_2c2)}"))

Now, we can calculate the Jacobians for the system. Note that we're mainly concerned about the center of mass for every link, so the D vector refers to the distances to the center of mass of each link.


In [ ]:
J_v1 = sp.simplify(sp.Matrix.hstack(
    R_01[:, 2].cross(R_01 @ D_1c1),
    sp.zeros(3, 1)
))

J_v2 = sp.simplify(sp.Matrix.hstack(
    R_01[:, 2].cross(R_01 @ D_1c2),
    R_02[:, 2].cross(R_02 @ D_2c2)
))

J_omega1 = sp.simplify(sp.Matrix.hstack(
    R_01[:, 2],
    sp.zeros(3, 1)
))

J_omega2 = sp.simplify(sp.Matrix.hstack(
    R_01[:, 2],
    R_02[:, 2]
))



In [ ]:
display(Math(f"J_{{v2}} = {sp.latex(J_v2)}"))
display(Math(f"J_{{\\omega2}} = {sp.latex(J_omega2)}"))

Using the moment of inertias for the system and the Jacobians, we'll calculate the required torque vector.


In [ ]:
M = (m_1 * J_v1.T @ J_v1 + (J_omega1.T * I_c1) @ J_omega1) + \
    (m_2 * J_v2.T @ J_v2 + (J_omega2.T * I_c2) @ J_omega2)

Q = sp.Matrix([theta_1, theta_2]).reshape(2, 1)
Q_d = sp.diff(Q, t)
Q_dd = sp.diff(Q_d, t)

V = M.diff(t) * Q_d - sp.Rational(1, 2) * sp.Matrix.vstack(
    Q_d.T * M.diff(theta_1) * Q_d,
    Q_d.T * M.diff(theta_2) * Q_d
)

P = m_1 * g * l_c1 * sp.sin(theta_1) + m_2 * g * l_1 * sp.sin(theta_1)

G = sp.Matrix([
    P.diff(theta_1),
    P.diff(theta_2)
]).reshape(2, 1)

T = sp.simplify(M * Q_dd + V + G)


In [ ]:
display(Math(f"M = {sp.latex(M)}"))

display(Math(f"Q = {sp.latex(Q)}"))
display(Math(f"Q_d = {sp.latex(Q_d)}"))
display(Math(f"Q_{{dd}} = {sp.latex(Q_dd)}"))

display(Math(f"V = {sp.latex(V)}"))

display(Math(f"G = {sp.latex(G)}"))

display(Math(f"T = {sp.latex(T)}"))

display(Math("M \\ddot{Q} + V + G = T"))


# Inverse Kinematics
By constraining the torque vector's first element to be zero, we can solve for the required thetas.


In [ ]:
sol = sp.solve(sp.Eq(T[0], 0), [theta_1, theta_1d, theta_1dd, theta_2, theta_2d, theta_2dd])

T_modified = sp.simplify(T.subs(theta_1dd, sol[0][2]))

In [ ]:
display(Math(f"solutions = {sp.latex(sol)}"))

display(Math(f"T_{{modified}} = {sp.latex(T_modified)}"))

# Forward Kinematics

In [ ]:
tau = sp.Symbol('tau')
system = [sp.Eq(T[0], 0).subs(l_c2, 0), sp.Eq(T[1], tau).subs(l_c2, 0)]
sol = sp.solve(system, [theta_1dd, theta_2dd])
display(sp.simplify(sol))